In [17]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from zoobot.shared.label_metadata import decals_all_campaigns_ortho_label_cols


In [18]:
dr12 = pd.read_parquet('/home/walml/repos/gz-decals-classifiers/data/catalogs/training_catalogs/dr12_ortho_v5_labelled_catalog.parquet')
dr5 = pd.read_parquet('/home/walml/repos/gz-decals-classifiers/data/catalogs/training_catalogs/dr5_ortho_v5_labelled_catalog.parquet')
dr8 = pd.read_parquet('/home/walml/repos/gz-decals-classifiers/data/catalogs/training_catalogs/dr8_ortho_v5_labelled_catalog.parquet')

In [19]:
dr12

,iauname,in_gzd_c,data_release,smooth-or-featured-dr12_total-votes,smooth-or-featured-dr12_smooth,smooth-or-featured-dr12_smooth_fraction,smooth-or-featured-dr12_featured-or-disk,smooth-or-featured-dr12_featured-or-disk_fraction,smooth-or-featured-dr12_artifact,smooth-or-featured-dr12_artifact_fraction,...,spiral-arm-count-dr5_3_fraction,spiral-arm-count-dr5_4_fraction,spiral-arm-count-dr5_more-than-4_fraction,spiral-arm-count-dr5_cant-tell_fraction,merging-dr5_none_fraction,merging-dr5_minor-disturbance_fraction,merging-dr5_major-disturbance_fraction,merging-dr5_merger_fraction,dr_with_most_votes,id_str
0,J161835.06+094825.4,False,DR1,47,29.0,0.617021,8.0,0.170213,10.0,0.212766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,387274_3520
1,J171558.33+294918.8,False,DR2,36,22.0,0.611111,4.0,0.111111,10.0,0.277778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,495330_3526
2,J152604.39+140215.0,True,DR2,38,28.0,0.736842,8.0,0.210526,2.0,0.052632,...,NaN,NaN,NaN,NaN,0.800000,0.0,0.0,0.200000,dr12,411197_3292
3,J074034.26+240659.4,False,DR2,38,17.0,0.447368,14.0,0.368421,7.0,0.184211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,465235_5856
4,J161921.36+131433.9,False,DR2,35,29.0,0.828571,1.0,0.028571,5.0,0.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,407046_2961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82355,J082618.08+111227.6,False,DR2,37,28.0,0.756757,2.0,0.054054,7.0,0.189189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,395315_1473
82356,J225716.71+000240.9,False,DR1,44,35.0,0.795455,5.0,0.113636,4.0,0.090909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,331745_4989
82357,J154806.17+142227.1,True,DR2,29,10.0,0.344828,17.0,0.586207,2.0,0.068966,...,NaN,NaN,NaN,NaN,0.833333,0.0,0.0,0.166667,dr12,412616_3932
82358,J001510.54-010434.1,False,DR1,49,39.0,0.795918,4.0,0.081633,6.0,0.122449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,324623_1219


In [20]:
megacatalog = pd.read_parquet('/home/walml/repos/decals-rings/results/rings_megacatalog.parquet', columns=['dr8_id', 'iauname', 'redshift'])
megacatalog.sample(5)

,dr8_id,iauname,redshift
6678886,488761_3260,None,0.277792
2441293,622433_1758,J121803.63+614527.7,0.107114
5945892,165340_6034,None,0.139267
8606621,477395_3226,None,0.156126
4997316,511049_648,None,0.185498


In [21]:
print(len(dr12))
dr12 = pd.merge(dr12, megacatalog, on='iauname', how='inner')
print(len(dr12))
# 85730
# 82387

82360
82360


In [22]:
print(len(dr5))
dr5 = pd.merge(dr5, megacatalog, on='iauname', how='inner')
print(len(dr5))
# 228059
# 222754

222754
222754


In [23]:
labelled = pd.concat([dr12, dr5, dr8], axis=0)
len(labelled)
# 365770

401250

Now make the unlabelled catalog from the remainder

In [24]:
unlabelled = megacatalog[~megacatalog['dr8_id'].isin(labelled['dr8_id'])]
unlabelled['id_str'] = unlabelled['dr8_id']
len(unlabelled)  # mostly high-z, some redshift filtering would be wise
# 8323600

/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


8593234

In [25]:
for col in decals_all_campaigns_ortho_label_cols:
    unlabelled[col] = 0
unlabelled = unlabelled.copy().sample(len(unlabelled), random_state=42)

/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [26]:
for df in [labelled, unlabelled]:  # dr8 already has but it points to the png
    df['dr8_id'] = df['id_str'] # ensure consistency
    df['brickid'] = df['id_str'].apply(lambda x: x.split('_')[0])
    df['objid'] = df['id_str'].apply(lambda x: x.split('_')[1])
    df['file_loc'] = '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/' + df['brickid'] + '/' + df['id_str'] + '.jpg'
    print(df['file_loc'][0])

/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.
/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmente

0    /share/nas2/walml/galaxy_zoo/decals/dr8/jpg/38...
0    /share/nas2/walml/galaxy_zoo/decals/dr8/jpg/30...
0    /share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...
Name: file_loc, dtype: object
/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/43543/43543_92.jpg


In [27]:
np.any(labelled[decals_all_campaigns_ortho_label_cols].isna())
# any(labelled[decals_all_campaigns_ortho_label_cols].isna()) doesn't work right

False

In [28]:
np.any(labelled[decals_all_campaigns_ortho_label_cols].isna(), axis=1).mean()
# any(labelled[decals_all_campaigns_ortho_label_cols].isna().values.max(axis=1)) doesn't work right

0.0

In [29]:
train_catalog, test_catalog = train_test_split(labelled, test_size=0.2, random_state=42)  # note that because we dropped a few rows which couldn't be cross-matched, this is not the same split as the original_images version

In [30]:
train_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz_desi/gz_desi_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz_desi/gz_desi_test_catalog.parquet'
unlabelled_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/desi_unlabelled/desi_unlabelled_catalog.parquet'

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

# this will not be uploaded as images, wayyyy to big
unlabelled.to_parquet(unlabelled_catalog_loc, index=False)

In [31]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    unlabelled_catalog_loc
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

64dcf77d1a67808e75eac32a8018f345
f3b554c4572e2fbba106d662b56523f1
37c9f07b5c058f55d515d2df08ff132a


In [32]:
# allow user to filter by redshift as needed